In [ ]:
!pip install datasets transformers
!apt install git-lfs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.4 MB/s 
     |████████████████████████████████| 4.2 MB 54.3 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.4 MB/s 
     |████████████████████████████████| 212 kB 26.9 MB/s 
     |████████████████████████████████| 140 kB 57.1 MB/s 
     |████████████████████████████████| 596 kB 52.4 MB/s 
     |████████████████████████████████| 127 kB 46.7 MB/s 
     |████████████████████████████████| 6.6 MB 7.4 MB/s 
     |████████████████████████████████| 144 kB 69.7 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 68.6 MB/s 
     |████████████████████████████████| 112 kB 58.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Utility Functions**

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset, load_metric
import datasets as ds
from pynvml import *
import numpy as np
from transformers import AutoTokenizer

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    if metric_name == "roc_auc":
      return metric.compute(prediction_scores=predictions, references=labels)
    if metric_name == "accuracy":
      return metric.compute(predictions=predictions, references=labels)

def eval_language(trainer,language):
    dataset = datasets["test.{}".format(language)]
    test_tokenized = dataset.map(preprocess_function, batched=True)
    evaluation = trainer.predict(test_tokenized)
    predictions = np.argmax(evaluation[0], axis=1)
    return (predictions == dataset["relavance_label"]).sum() / predictions.shape[0]

def preprocess_function(examples):
    examples["paragraphs"] = ["{} - {}".format(title,content) for title,content in zip(examples["web_page_title"],examples["web_page_snippet"])]
    return tokenizer(examples["query"], examples["paragraphs"], padding = True, truncation=True)

# **Parameters for Web Page Ranking training**

In [ ]:
#model_checkpoint = "/content/gdrive/MyDrive/mnli"
#model_checkpoint = "distilbert-base-multilingual-cased"
model_checkpoint = "/content/gdrive/MyDrive/thesis/baseline_intermediate/xquad"

metric_name = "accuracy"
metric = load_metric(metric_name)
batch_size = 64
num_labels = 5
datasets = load_dataset("xglue","wpr")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#tokenized_datasets = datasets.map(preprocess_function,batched=True)
#tokenized_datasets = tokenized_datasets.rename_column("relavance_label","label")
#tokenized_datasets = tokenized_datasets.rename_column("relavance_label","label")

Generating train split:   0%|          | 0/99997 [00:00<?, ? examples/s]

Generating validation.en split:   0%|          | 0/10008 [00:00<?, ? examples/s]

Generating validation.de split:   0%|          | 0/10004 [00:00<?, ? examples/s]

Generating validation.es split:   0%|          | 0/10004 [00:00<?, ? examples/s]

Generating validation.fr split:   0%|          | 0/10005 [00:00<?, ? examples/s]

...

Generating test.pt split:   0%|          | 0/10015 [00:00<?, ? examples/s]

Generating test.zh split:   0%|          | 0/9999 [00:00<?, ? examples/s]

Dataset x_glue downloaded and prepared to /root/.cache/huggingface/datasets/x_glue/wpr/1.0.0/8566eedecd9ab28e01c051c023dadf97bf408e5195f76b06aba70ebd4697ae08. Subsequent calls will reuse this data.


  0%|          | 0/15 [00:00<?, ?it/s]

Didn't find file /content/gdrive/MyDrive/thesis/baseline_intermediate/xquad/added_tokens.json. We won't load it.
loading file /content/gdrive/MyDrive/thesis/baseline_intermediate/xquad/vocab.txt
loading file /content/gdrive/MyDrive/thesis/baseline_intermediate/xquad/tokenizer.json
loading file None
loading file /content/gdrive/MyDrive/thesis/baseline_intermediate/xquad/special_tokens_map.json
loading file /content/gdrive/MyDrive/thesis/baseline_intermediate/xquad/tokenizer_config.json


# **Training Configuration**

In [ ]:
from transformers import TrainingArguments, Trainer,AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,ignore_mismatched_sizes=True).to("cuda")
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-WPR",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    push_to_hub=False,
    fp16 = True
)

Some weights of the model checkpoint at /content/gdrive/MyDrive/thesis/baseline_intermediate/xquad were not used when initializing DistilBertForSequenceClassification: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/gdrive/MyDrive/thesis/baseline_intermediate/xquad and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You shoul

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation.en"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("wpr-train-en")

Using amp half precision backend
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: web_page_title, query, paragraphs, web_page_snippet. If web_page_title, query, paragraphs, web_page_snippet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 99997
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 4689


Epoch,Training Loss,Validation Loss,Accuracy
1,1.140500,1.161697,0.482914
2,1.048100,1.199411,0.464728
3,0.973400,1.187830,0.480715


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: web_page_title, query, paragraphs, web_page_snippet. If web_page_title, query, paragraphs, web_page_snippet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10008
  Batch size = 64
Saving model checkpoint to xquad-finetuned-WPR/checkpoint-1563
Configuration saved in xquad-finetuned-WPR/checkpoint-1563/config.json
Model weights saved in xquad-finetuned-WPR/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in xquad-finetuned-WPR/checkpoint-1563/tokenizer_config.json
Special tokens file saved in xquad-finetuned-WPR/checkpoint-1563/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: web_page_title, query, 

In [ ]:
!cp -r '/content/wpr-train-en' '/content/gdrive/MyDrive/thesis/trained_intermediate/xquad_wpr'

# **Evaluation of the model**

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer,AutoModelForSequenceClassification,AutoTokenizer

model_checkpoints = ['/content/gdrive/MyDrive/thesis/baseline_downstream/wpr-train-en'
]
languages = ['en', 'de', 'es', 'fr', 'it', 'pt', 'zh']
results = {}
for model_checkpoint in model_checkpoints:
  results[model_checkpoint] = {}
  model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  trainer = Trainer(
      model,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  for language in languages:
    result = eval_language(trainer,language)
    results[model_checkpoint][language] = result



loading configuration file /content/gdrive/MyDrive/thesis/baseline_downstream/wpr-train-en/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/gdrive/MyDrive/thesis/baseline_downstream/wpr-train-en",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_ver

  0%|          | 0/11 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: query, web_page_title, paragraphs, web_page_snippet, relavance_label. If query, web_page_title, paragraphs, web_page_snippet, relavance_label are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10004
  Batch size = 8


KeyboardInterrupt: ignored

In [ ]:
results